In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Output, Dropdown, SelectMultiple, HBox, VBox, Button, IntSlider, FloatRangeSlider, FloatSlider, Text, Textarea, Combobox, HBox, SelectMultiple
from IPython.display import display

from surianalytics.connectors import RESTSciriusConnector
from surianalytics.datamining import min_max_scaling

import pandas as pd
import numpy as np

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)

In [ ]:
c = RESTSciriusConnector()
c.set_page_size(1000)

### Timestamp handlers

In [ ]:
from datetime import datetime, timedelta

In [ ]:
TIME_END = datetime.utcnow()
TIME_BEGINNING = TIME_END - timedelta(minutes=60)

SLIDER_TIME_MINUTES = IntSlider(
    value=60,
    min=5,
    max=600,
    step=5,
    description='Minutes',
    orientation='horizontal',
    readout=True,
)

In [ ]:
# TimePicker is not in ipywidgests 7.7
# Use text boxes instead for now
TEXT_TIME_BEGINNING = Text(
    description="From: ",
    value=TIME_BEGINNING.isoformat(),
    continuous_update=True
)
TEXT_TIME_END = Text(
    description="To: ",
    value=TIME_END.isoformat(),
    continuous_update=True
)
_ = c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
def handler_update_timeframe(args):
    c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
BUTTON_UPDATE_TIME = Button(description="Set time")
BUTTON_UPDATE_TIME.on_click(handler_update_timeframe)

In [ ]:
def handler_reset_timeframe(args):
    time_to = datetime.utcnow()
    time_from = time_to - timedelta(minutes=SLIDER_TIME_MINUTES.value)
    TEXT_TIME_END.value = time_to.isoformat()
    TEXT_TIME_BEGINNING.value = time_from.isoformat()
    handler_update_timeframe()

In [ ]:
BUTTON_RESET_TIME = Button(description="Generate time")
BUTTON_RESET_TIME.on_click(handler_reset_timeframe)

In [ ]:
BOX_TIME = HBox([SLIDER_TIME_MINUTES, BUTTON_RESET_TIME, TEXT_TIME_BEGINNING, TEXT_TIME_END, BUTTON_UPDATE_TIME])

In [ ]:
BOX_TIME

### Filters documentation

#### SMB

```
event_type: smb
```

#### Alerts

```
event_type: alert
```

#### HTTP

Request to HTTP URL with potential malicious suffix. Requires Stamus SSP network enrichment.

```
event_type: http AND (http.url: *zip OR http.url: *.exe) AND net_info.src_agg: * AND NOT net_info.dest_agg: *
```

#### IoC

Bad IP, either side connection.

```
dest_ip: 37.157.114.70 OR src_ip: 37.157.114.70
```

IoC lookup.

```
(edr: * AND *.ps1) OR http.url: */www/ust.zip
```

### Unique fields exploration

In [ ]:
TEXT_QUERY = Textarea(
    description='Query filter:',
    value="*",
    continuous_update=True
)
TEXT_QUERY

In [ ]:
SELECT_FIELD = Combobox(options=c.get_unique_fields(), value="flow_id")
SELECT_FIELD

In [ ]:
DF = pd.DataFrame()
OUTPUT_UNIQUE = Output()
def handler_pull_data(args):
    global DF
    DF = pd.DataFrame(c.get_eve_unique_values(counts="yes", field=SELECT_FIELD.value, qfilter=TEXT_QUERY.value))
    OUTPUT_UNIQUE.clear_output()
    with OUTPUT_UNIQUE:
        display(DF.sort_values(by="key"))

In [ ]:
BUTTON_PULL_UNIQUE = Button(description="Pull unique values")
BUTTON_PULL_UNIQUE.on_click(handler_pull_data)

In [ ]:
def handler_show_simple_uniq(args):
    OUTPUT_UNIQUE.clear_output()
    with OUTPUT_UNIQUE:
        print("\n".join(sorted(list(DF.key.unique()))))

In [ ]:
c.last_request

In [ ]:
BUTTON_SHOW_SIMPLE = Button(description="Show simple values")
BUTTON_SHOW_SIMPLE.on_click(handler_show_simple_uniq)

In [ ]:
display(BUTTON_PULL_UNIQUE)
display(BUTTON_SHOW_SIMPLE)

In [ ]:
display(OUTPUT_UNIQUE)

### Explore events

In [ ]:
import copy

In [ ]:
DEFAULT_COLUMNS = ["timestamp", "flow_id", "event_type"]
COLS = copy.deepcopy(DEFAULT_COLUMNS)

In [ ]:
OUTPUT_DEBUG = Output()

In [ ]:
DF_EVENTS = pd.DataFrame(columns=DEFAULT_COLUMNS)

In [ ]:
SELECT_COLUMNS = Combobox(options=sorted(list(DF_EVENTS.dropna(how="all", axis=1).columns.values)), description="Columns")

In [ ]:
SELECT_AGG_COLUMN = Combobox(options=COLS, value="flow_id", description="Group by")

In [ ]:
def handler_add_col(args):
    OUTPUT_DEBUG.clear_output()
    with OUTPUT_DEBUG:
        global COLS
        col = SELECT_COLUMNS.value
        if col in list(DF_EVENTS.columns.values) and col not in COLS:
            COLS.append(col)
            SELECT_AGG_COLUMN.options = COLS
            print(COLS)
        else:
            print("{} already in selected columns".format(col))

In [ ]:
BUTTON_ADD_COL = Button(description="Add column")
BUTTON_ADD_COL.on_click(handler_add_col)

In [ ]:
def handler_clear_col(args):
    OUTPUT_DEBUG.clear_output()
    with OUTPUT_DEBUG:
        global COLS
        COLS = DEFAULT_COLUMNS
        SELECT_AGG_COLUMN.options = DEFAULT_COLUMNS
        print("columns are now {}".format(COLS))

In [ ]:
BUTTON_CLEAR_COL = Button(description="Clear columns")
BUTTON_CLEAR_COL.on_click(handler_clear_col)

In [ ]:
OUTPUT_EVE = Output()

In [ ]:
def handler_pull_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        global DF_EVENTS
        DF_EVENTS = c.get_events_df(qfilter=TEXT_QUERY.value)
        SELECT_COLUMNS.options = list(DF_EVENTS.columns.values)
        print("downloaded {} rows".format(len(DF_EVENTS)))

In [ ]:
BUTTON_EVE_PULL = Button(description="Download EVE")
BUTTON_EVE_PULL.on_click(handler_pull_events)

In [ ]:
def handler_aggregate_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        df = (
            DF_EVENTS
                .sort_values(by="timestamp")
                .dropna(axis=1, how="all")
                .groupby(SELECT_AGG_COLUMN.value)
                .agg({item: ["min", "max"] if item == "timestamp" else ["unique"] for item in COLS if item != SELECT_AGG_COLUMN.value})
        )
        display(df)

In [ ]:
BUTTON_EVE_AGG = Button(description="Aggregate EVE")
BUTTON_EVE_AGG.on_click(handler_aggregate_events)

In [ ]:
def handler_show_events(args):
    OUTPUT_EVE.clear_output()
    with OUTPUT_EVE:
        df = (
            DF_EVENTS[COLS]
                .sort_values(by="timestamp")
                .head(500)
                .dropna(axis=1, how="all")
        )
        display(df)

In [ ]:
BUTTON_EVE_SHOW = Button(description="Show EVE")
BUTTON_EVE_SHOW.on_click(handler_show_events)

In [ ]:
BOX_EVE = HBox([BUTTON_EVE_PULL, BUTTON_EVE_SHOW, BUTTON_EVE_AGG])
BOX_ADD_COL = HBox([SELECT_COLUMNS, BUTTON_ADD_COL, BUTTON_CLEAR_COL])

display(SELECT_AGG_COLUMN)

display(BOX_ADD_COL)
display(BOX_EVE)

In [ ]:
display(OUTPUT_DEBUG)
display(OUTPUT_EVE)